In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


We focus on the 406 k individuals in one of the following set and prepare `split_400k` column


In [24]:
phe_in_f="/scratch/groups/mrivas/ukbb24983/phenotypedata/master_phe/master.20211020.phe.zst"
phe_out_f<- '/scratch/groups/mrivas/projects/PRS/private_output/20211022_400k/20211022.phe'
phe_keep_f<- '/scratch/groups/mrivas/projects/PRS/private_output/20211022_400k/20211022.keep.phe'

In [5]:
'GBE_ID_list.txt' %>% fread(head=F) %>% pull() -> GBE_IDs


In [6]:
cols <- c('#FID', 'IID', 'population', 'split', 'split_nonWB', 'age', 'sex', paste0('Global_PC', 1:18), GBE_IDs)

In [16]:
fread(
    cmd=paste('zstdcat', phe_in_f),
    colClasses = c('#FID'='character', 'IID'='character'),
    select=cols
) %>%
rename_with(function(x){str_replace(x, '#', '')}, starts_with("#")) -> phe_df


In [17]:
phe_df %>%
mutate(
    split_400k = case_when(
        population == 'white_british' ~  split,
        population != 'related' ~ split_nonWB,
        TRUE ~ 'NA'
    )
) %>%
replace_na(list(split_400k = 'NA')) -> phe_split400k_df


In [23]:
phe_split400k_df %>% 
rename('#FID' = 'FID') %>%
fwrite(phe_out_f, sep='\t', na = "NA", quote=F)


In [26]:
phe_split400k_df %>% filter(split_400k != 'NA') %>%
select(FID, IID) %>%
fwrite(phe_keep_f, sep='\t', na = "NA", quote=F, col.names = FALSE)


In [20]:
phe_split400k_df %>% filter(split_400k != 'NA') %>% nrow

[1] 406722

In [19]:
phe_split400k_df %>%
count(split_400k)


,split_400k,n
,<chr>,<int>
1,NA,110047
2,test,81346
3,train,284703
4,val,40673


In [18]:
phe_split400k_df %>%
count(split_400k, population)


split_400k,population,n
<chr>,<chr>,<int>
NA,-9,28497
NA,DO_NOT_PASS_SQC,1826
NA,related,44632
NA,NA,35092
test,african,1300
test,e_asian,341
test,non_british_white,4981
test,others,5732
test,s_asian,1567
